# Generating Word Embeddings - Lab

## Introduction

In this lab, we'll learn how to generate our own word embeddings by training our own Word2Vec model, and also by building embedding layers right into our Deep Neural Networks!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of word2vec such as vector size and window size

## Getting Started

In this lab, we'll start by creating our own word embeddings by making use of the Word2Vec Model. Then, we'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in our model. 

The easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which we can use to begin training immediately. For this lab, we'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.  In this lab, we'll learn how to train a Word2Vec model on the text data to generate word embeddings for them. In the next lab, we'll then use the vectors created by our Word2Vec model to effectively train a classifier to predict the category of news given the headline and description of each article. In this lab, we won't do any classification, although we will learn how to train a Word2Vec model and explore the relationships between different word vectors in our embedding!

Run the cell below to import everything we'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, we'll import the data. You'll find the data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to also include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what our data looks like. 

In [7]:
raw_df = pd.read_json('News_Category_Dataset_v2.json', lines = True)
raw_df.info()
raw_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
category             200853 non-null object
headline             200853 non-null object
authors              200853 non-null object
link                 200853 non-null object
short_description    200853 non-null object
date                 200853 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since we're working with text data, we'll still need to do some basic preprocessing and tokenize our data. You'll notice from the sample of the data above that two different columns contain text data--`headline` and `short_description`. The more text data our Word2Vec model has, the better it will perform. Therefore, we'll want to combine the two columns before tokenizing each comment and training our Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [11]:
raw_df['combined_text'] = raw_df.headline + ' ' + raw_df.short_description
data = raw_df.combined_text.map(word_tokenize)

Let's inspect the first 5 items in `data` to see how everything looks. 

In [12]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

You'll notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Recall from our previous lesson on how Word2Vec works that we can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If our window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example we saw).

Now that we've prepared our data, let's train our model and explore a bit!

## Training the Model

We'll start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [13]:
model = Word2Vec(data, size = 100, window = 5, min_count = 1, workers = 4)

Now, that we've created our Word2Vec model, we still need to train it on our model. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which we can find in `model.corpus_count`. 
    * The number of `epochs` we want to train for, which we'll set to `10`

In [14]:
model.train(data, total_examples = model.corpus_count, epochs = 10)

(55566165, 67352790)

Great! We now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which we'll find stored inside of `model.wv`. For simplicity's sake, let's go ahead and store this inside of the variable `wv` in order to save ourselves some keystrokes down the line. 

In [15]:
wv = model.wv

## Examining Our Word Vectors

Now that we have a trained Word2Vec model, let's go ahead and explore the relationships between some of the words in our corpus! 

One cool thing we can use Word2Vec for is to get the most similar words to a given word. We can do this passing in the word to `wv.most_similar()`. 

In the cell below, let's try getting the most similar word to `'Texas'`.

In [16]:
wv.most_similar('Texas')

[('Oklahoma', 0.821514368057251),
 ('Maryland', 0.8208914995193481),
 ('Pennsylvania', 0.8186578154563904),
 ('Ohio', 0.8171935081481934),
 ('Georgia', 0.8164858818054199),
 ('Louisiana', 0.8147978782653809),
 ('Connecticut', 0.8056623935699463),
 ('Arkansas', 0.7968416213989258),
 ('Florida', 0.79017573595047),
 ('California', 0.7861220240592957)]

Interesting! All of the most similar words are also states. 

We can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [17]:
wv.most_similar(negative = 'Texas')

[('Parent/Grandparent', 0.4507206082344055),
 ('End-of-School', 0.4138534367084503),
 ('Slammers', 0.3796992599964142),
 ('Optimizing', 0.3774212598800659),
 ('Jobseekers', 0.3737824857234955),
 ('Begets', 0.3694300353527069),
 ('Uglies', 0.3667285740375519),
 ('Per-Mile', 0.3642973303794861),
 ('PROVINCETOWN', 0.361206591129303),
 ('uh-oh', 0.3583786189556122)]

These seem like just noise. This is because of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with our given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

We can also get the vector for a given word by passing in the word as if we were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [18]:
wv['Texas']

array([ 1.3941301e-01,  1.2018921e+00,  9.1002655e-01,  7.6049227e-01,
       -2.4775276e+00,  2.6847365e+00,  7.5587523e-01,  1.0383323e+00,
       -1.3331070e+00,  2.5565462e+00,  1.7452667e+00,  2.1962974e+00,
        2.9807579e-02, -7.0086038e-01,  1.2094835e+00, -1.4157410e+00,
       -3.6924775e+00, -5.6172544e-01,  1.3493964e-01, -1.0302271e+00,
       -6.8651366e-01, -1.9812798e-02,  1.3800000e+00,  1.6096298e+00,
        3.1144133e+00, -1.9999999e+00, -1.8413593e+00,  6.0246259e-01,
       -3.4429681e-01, -3.1665802e-01,  8.9094609e-01,  1.1572460e+00,
       -1.4055431e-01,  7.1913785e-01, -1.0538394e+00, -1.3602791e+00,
        7.6170176e-01, -1.1608320e+00, -8.6895287e-01, -2.0398109e+00,
       -1.3926603e+00, -1.5369834e+00, -1.7800190e+00, -1.0130492e+00,
       -1.3094934e+00, -5.5337852e-01, -1.5396984e+00, -1.3719521e+00,
       -2.7077296e-01, -7.2475368e-01, -1.8829529e+00, -1.5954462e+00,
        1.9671875e+00,  6.0426313e-01,  1.0237604e+00, -1.1676887e+00,
      

Let's get all of the word vectors from the object at once. We can find these inside of `wv.vectors`.  Do this now in the cell below.  

In [19]:
wv.vectors

array([[-2.0946176e+00, -3.7877396e-02, -1.4922525e+00, ...,
        -8.1801343e-01,  3.5270432e-01,  5.9564382e-01],
       [-8.1105846e-01, -9.1879529e-01, -2.1255341e+00, ...,
        -1.9366144e+00, -1.6064601e+00,  4.9059033e-01],
       [-1.4926424e+00, -4.8938605e-01, -1.6212455e+00, ...,
        -7.4821569e-02, -1.3681468e+00, -2.1418150e-01],
       ...,
       [ 2.6753360e-02,  6.9601186e-02, -2.0494884e-02, ...,
         3.9484676e-02,  3.7051078e-02,  6.8736926e-02],
       [-5.2289262e-02,  1.1119985e-03, -5.5426743e-02, ...,
        -4.1769408e-02,  4.4464272e-02, -1.0464698e-02],
       [ 4.2285722e-02,  3.0675525e-02, -4.2130042e-02, ...,
         5.9332363e-02,  5.0661732e-03,  9.9896044e-02]], dtype=float32)

As a final exercise, let's try recreating the _'king' - 'man' + 'woman' = 'queen'_ example we've seen before. We can do this by using the `most_similar` function and putting the things we want added together inside of an array passed to the `positive` parameter, and the things we want subtracted as an array passed to the the `negative` parameter. 

Do this now in the cell below. 

In [27]:
wv.most_similar(positive = ['king','woman'], negative = ['man'])

[('princess', 0.5812712907791138),
 ('villain', 0.5657402276992798),
 ('queen', 0.5652158260345459),
 ('symbol', 0.5616866946220398),
 ('coin', 0.558171272277832),
 ('brunette', 0.5574144124984741),
 ('revival', 0.5519267320632935),
 ('crown', 0.5462484359741211),
 ('title', 0.5459083318710327),
 ('monster', 0.5452340245246887)]

As we can see from the output above, our model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As we can see from the word in first place, 'reminiscent', our model is far from perfect. This is likely because we didn't give it too much training, or training data. However, for the small amount of training data it was given, the model still performs remarkably well! 

We'll see in the next lab that from a practical standpoint, one of the best things we can do for performance is to start by loading in the weights from an open-sourced model that has been trained for a very long time on a massive amount of data, such as the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless our text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, we learned how to train and use a Word2Vec model to created vectorized word embeddings!